In [2]:
# --- Imports ---
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import preprocess as pre
import time

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

import xgboost as xgb

In [3]:
# --- Data Preprocessing --- 
start = time.time()
pre.ais_trainCleanup("../CSV/big_files/ais_train.csv", "ais_train_processed.csv")
end = time.time()

print(end-start)

c:\Users\marti\Desktop\ML-master\Kuking\preprocess.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boat["time"] = boat["time"].astype("datetime64[ns]")
c:\Users\marti\Desktop\ML-master\Kuking\preprocess.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boat["drift"] = boat["cog"] - boat["heading"]
c:\Users\marti\Desktop\ML-master\Kuking\preprocess.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

TypeError: '>=' not supported between instances of 'Timedelta' and 'Timestamp'